This feature is experiment and useful in the legacy notebook.  It would be awesome to track markdown to code conversions.

In [ ]:
        try:
            from .run import run_cell
            from .template import template
            from .literate import literate, rstrip
        except:
            from run import run_cell
            from template import template
            from literate import literate, rstrip
        from json import dumps
        from IPython.display import display, Javascript, Markdown
        from jsonpatch import make_patch
        from toolz.curried import pipe, compose, do

In [ ]:
        get_selector = """window.get_selector = function(element){if(window.Jupyter){
        return Jupyter.notebook.get_cell(
            Jupyter.notebook.get_cell_elements()
                .map(function(i, x){return ($(x).is(element) && i) || -1;})
                .filter(function(i, x){return x>-1;})[0]);}}"""

In [ ]:
        set_metadata = """get_selector(this.selector.parent()).metadata["patch"] = {};"""

In [ ]:
        def split_and_dict(text):
            return {'source': text.splitlines()}

In [ ]:
        def runner(text):
            code = pipe(text, template, do(compose(display, Markdown)), literate.render)
            pipe(
                make_patch(*map(split_and_dict, (text, code))).patch,
                dumps, set_metadata.format, Javascript, display)
            return code

In [ ]:
        def load_ipython_extension(ip=__import__('IPython').get_ipython()):
            display(Javascript(get_selector))
            run_cell([runner])(ip)